# Modelado — Próximo pedido **DIGITAL** (v3, 100% PySpark + Backtesting & Ensembles)

**Objetivo:** predecir si el **próximo pedido** de un cliente será **DIGITAL**.

**Mejoras sobre v2:**

- Etiqueta *determinista* por cliente-mes.

- **Features avanzadas** (rolling 3m, crecimiento, ciclo de vida del cliente).

- **Priorización temporal**: split por fecha y opción de **backtesting**.

- **Modelos**: Regresión Logística, **Random Forest**, **GBT** (boosting).

- **Búsqueda de hiperparámetros** y **balanceo por clase**.

- **Tuning de umbral**, **métricas por segmento** y **calibración** de prob.

- Exportación de **artefactos** y **métricas** para el repo.


## 1) Sesión Spark y parámetros

- Ejecuta local con todos los cores.

- Ajusta `DATA_DIR` si tu dataset está en otra ruta.

- `TEST_START_YM` define el **corte temporal** principal.

- `BACKTEST_SPLITS` permite evaluar **varios cortes** (opcional).


In [ ]:
from pyspark.sql import SparkSession, functions as F, types as T, Window
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, Imputer
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder
from pyspark.ml.functions import vector_to_array

spark = (
    SparkSession.builder
    .appName("modelado-proximo-pedido-digital-v3")
    .master("local[*]")
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")
    .config("spark.driver.memory", "4g")
    .config("spark.sql.warehouse.dir", "./spark-warehouse")
    .getOrCreate()
)
spark.sparkContext.setLogLevel("WARN")

DATA_DIR = "dataset/dataset"      # <--- Ajusta si tu ruta cambia
TEST_START_YM = "2024-01"         # Entrena con ym < TEST_START_YM | Evalúa con ym >= TEST_START_YM
BACKTEST_SPLITS = ["2023-08", "2023-10", "2023-12", "2024-01"]  # Opcional

print("Spark version:", spark.version)

## 2) Carga y preparación mínima

- Derivamos `month_first`, `ym` y `is_digital`.

- Mantenemos **nombres simples** y tipos adecuados.


In [ ]:
df = spark.read.parquet(DATA_DIR)

expected = [
    "cliente_id","pais_cd","region_comercial_txt","agencia_id","ruta_id",
    "tipo_cliente_cd","madurez_digital_cd","estrellas_txt","frecuencia_visitas_cd",
    "fecha_pedido_dt","canal_pedido_cd","facturacion_usd_val",
    "materiales_distintos_val","cajas_fisicas"
]
present = [c for c in expected if c in df.columns]
print("Columnas presentes:", present)

df = (df
    .withColumn("month_first", F.trunc("fecha_pedido_dt", "month"))
    .withColumn("ym", F.date_format("month_first", "yyyy-MM"))
    .withColumn("is_digital", F.when(F.col("canal_pedido_cd")=="DIGITAL", 1).otherwise(0))
)

df.select("cliente_id","fecha_pedido_dt","ym","canal_pedido_cd","is_digital").show(5, truncate=False)

## 3) Etiqueta por **cliente-mes** (determinista, sin fuga)

- Orden por `fecha_pedido_dt` (y *tie-break* con hash de todas las columnas).

- Para cada mes de un cliente, tomamos el **último pedido** y definimos `label` como si el **próximo pedido** del cliente es DIGITAL.

- Agregamos `recency_days_last` (días desde el pedido previo).


In [ ]:
all_cols = df.columns
w_client_order = Window.partitionBy("cliente_id").orderBy(F.col("fecha_pedido_dt").asc(),
                                                          F.hash(*[F.col(c) for c in all_cols]).asc())
w_client_month_desc = Window.partitionBy("cliente_id","month_first").orderBy(F.col("fecha_pedido_dt").desc(),
                                                                             F.hash(*[F.col(c) for c in all_cols]).desc())

orders = (df
    .withColumn("prev_dt", F.lag("fecha_pedido_dt").over(w_client_order))
    .withColumn("next_canal", F.lead("canal_pedido_cd").over(w_client_order))
    .withColumn("next_is_digital", F.when(F.col("next_canal")=="DIGITAL", 1).otherwise(0))
    .withColumn("recency_days", F.datediff(F.col("fecha_pedido_dt"), F.col("prev_dt")))
    .withColumn("rn_month_desc", F.row_number().over(w_client_month_desc))
)

last_in_month = (orders
    .filter(F.col("rn_month_desc")==1)
    .select("cliente_id","month_first","ym",
            F.col("recency_days").alias("recency_days_last"),
            F.col("next_is_digital").alias("label"))
)

last_in_month.show(5, truncate=False)

## 4) Feature engineering (RFM + rolling + ciclo de vida + priors de segmento)

**Señales por cliente-mes**:

- Volumen/valor: `n_orders`, `sum_fact`, `avg_fact`, `sum_cajas`, `avg_cajas`, `avg_mat_dist`.

- Comportamiento: `digital_ratio` del mes, `lag1_digital_ratio`, **rolling 3m** y `growth_digital_ratio`.

- Ciclo de vida: meses desde primer pedido (`months_since_first`).

- Priors de segmento (históricos): `region_digital_ratio_lag1`, `tipo_cliente_digital_ratio_lag1`.

*(las razones/ratios de segmento se calculan por mes y se desplazan un mes hacia atrás para evitar fuga)*


In [ ]:
# Agregados cliente-mes
monthly_agg = (df.groupBy("cliente_id","month_first","ym")
    .agg(
        F.count("*").alias("n_orders"),
        F.avg("is_digital").alias("digital_ratio"),
        F.sum(F.col("facturacion_usd_val").cast("double")).alias("sum_fact"),
        F.avg(F.col("facturacion_usd_val").cast("double")).alias("avg_fact"),
        F.sum(F.col("cajas_fisicas").cast("double")).alias("sum_cajas"),
        F.avg(F.col("cajas_fisicas").cast("double")).alias("avg_cajas"),
        F.avg(F.col("materiales_distintos_val").cast("double")).alias("avg_mat_dist"),
        F.first("tipo_cliente_cd", ignorenulls=True).alias("tipo_cliente_cd"),
        F.first("madurez_digital_cd", ignorenulls=True).alias("madurez_digital_cd"),
        F.first("frecuencia_visitas_cd", ignorenulls=True).alias("frecuencia_visitas_cd"),
        F.first("pais_cd", ignorenulls=True).alias("pais_cd"),
        F.first("region_comercial_txt", ignorenulls=True).alias("region_comercial_txt")
    )
)

# Ciclo de vida: meses desde primer pedido
w_client_month = Window.partitionBy("cliente_id").orderBy(F.col("month_first").asc())
first_month = (monthly_agg
               .withColumn("first_month", F.first("month_first", ignorenulls=True).over(w_client_month))
               .select("cliente_id","first_month").distinct())

monthly_agg = monthly_agg.join(first_month, on="cliente_id", how="left")                          .withColumn("months_since_first", F.floor(F.months_between("month_first", "first_month")))

# Ratios por segmento (mes a mes) y lag para evitar fuga
region_month = (df.groupBy("region_comercial_txt","month_first")
                  .agg(F.avg("is_digital").alias("region_digital_ratio")))
region_month = region_month.withColumn("ym", F.date_format("month_first", "yyyy-MM"))
w_region = Window.partitionBy("region_comercial_txt").orderBy(F.col("month_first").asc())
region_month = region_month.withColumn("region_digital_ratio_lag1", F.lag("region_digital_ratio", 1).over(w_region))                            .select("region_comercial_txt","ym","region_digital_ratio_lag1")

tipo_month = (df.groupBy("tipo_cliente_cd","month_first")
                .agg(F.avg("is_digital").alias("tipo_digital_ratio")))
tipo_month = tipo_month.withColumn("ym", F.date_format("month_first", "yyyy-MM"))
w_tipo = Window.partitionBy("tipo_cliente_cd").orderBy(F.col("month_first").asc())
tipo_month = tipo_month.withColumn("tipo_digital_ratio_lag1", F.lag("tipo_digital_ratio", 1).over(w_tipo))                        .select("tipo_cliente_cd","ym","tipo_digital_ratio_lag1")

# Construcción del dataset con label
w_roll3 = w_client_month.rowsBetween(-3, -1)  # 3 meses previos
ds = (monthly_agg
    .join(last_in_month, on=["cliente_id","month_first","ym"], how="left")
    .withColumn("lag1_digital_ratio", F.lag("digital_ratio", 1).over(w_client_month))
    .withColumn("n_orders_3m", F.sum("n_orders").over(w_roll3))
    .withColumn("digital_ratio_3m", F.avg("digital_ratio").over(w_roll3))
    .withColumn("sum_fact_3m", F.sum("sum_fact").over(w_roll3))
    .withColumn("growth_digital_ratio", F.col("digital_ratio") - F.col("lag1_digital_ratio"))
    .join(region_month, on=["region_comercial_txt","ym"], how="left")
    .join(tipo_month, on=["tipo_cliente_cd","ym"], how="left")
    .filter(F.col("label").isNotNull())
)

ds.select("cliente_id","ym","n_orders","digital_ratio","lag1_digital_ratio",
          "n_orders_3m","digital_ratio_3m","growth_digital_ratio",
          "months_since_first","region_digital_ratio_lag1","tipo_digital_ratio_lag1",
          "label").show(5, truncate=False)

## 5) Split temporal y **balance de clases**

- Train: `ym < TEST_START_YM`.

- Test:  `ym >= TEST_START_YM`.

- Mostramos la **distribución de la etiqueta** y aplicamos **weightCol** cuando hay desbalance.


In [ ]:
train = ds.filter(F.col("ym") < F.lit(TEST_START_YM))
test  = ds.filter(F.col("ym") >= F.lit(TEST_START_YM))

print("Train rows:", train.count(), " | Test rows:", test.count())
print("\nDistribución de la etiqueta:")
for name, d in [("train", train), ("test", test)]:
    print(f"--- {name} ---")
    d.groupBy("label").count().orderBy("label").show()

## 6) Pipeline de modelado (reutilizable)

- Imputación para numéricas, Indexación+OneHot para categóricas, `VectorAssembler`.

- Modelos soportados: `lr`, `rf`, `gbt`.

- **TrainValidationSplit** con *grid* compacto.

- **weightCol** para balancear.


In [ ]:
num_cols = ["n_orders","digital_ratio","lag1_digital_ratio","sum_fact","avg_fact",
            "sum_cajas","avg_cajas","avg_mat_dist","recency_days_last",
            "n_orders_3m","digital_ratio_3m","sum_fact_3m","growth_digital_ratio",
            "months_since_first","region_digital_ratio_lag1","tipo_digital_ratio_lag1"]

cat_cols = ["tipo_cliente_cd","madurez_digital_cd","frecuencia_visitas_cd","pais_cd","region_comercial_txt"]

imputer = Imputer(inputCols=num_cols, outputCols=[c + "_imp" for c in num_cols])
indexers = [StringIndexer(inputCol=c, outputCol=c + "_idx", handleInvalid="keep") for c in cat_cols]
encoders = [OneHotEncoder(inputCols=[c + "_idx"], outputCols=[c + "_oh"]) for c in cat_cols]
feature_cols = [c + "_imp" for c in num_cols] + [c + "_oh" for c in cat_cols]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

def build_and_fit(model_name: str, train_df):
    # Balanceo simple
    pos = train_df.filter(F.col("label")==1).count()
    neg = train_df.filter(F.col("label")==0).count()
    balancing_ratio = neg / float(max(pos, 1)) if pos else 1.0
    train_w = train_df.withColumn("weight", F.when(F.col("label")==1, F.lit(balancing_ratio)).otherwise(F.lit(1.0)))

    if model_name == "lr":
        clf = LogisticRegression(featuresCol="features", labelCol="label", weightCol="weight",
                                 maxIter=80, regParam=0.01, elasticNetParam=0.0)
        paramGrid = (ParamGridBuilder()
                     .addGrid(clf.regParam, [0.0, 0.01, 0.1])
                     .addGrid(clf.elasticNetParam, [0.0, 0.5, 1.0])
                     .build())
    elif model_name == "rf":
        clf = RandomForestClassifier(featuresCol="features", labelCol="label", weightCol="weight",
                                     numTrees=200, maxDepth=10, featureSubsetStrategy="sqrt",
                                     subsamplingRate=0.8, seed=42)
        paramGrid = (ParamGridBuilder()
                     .addGrid(clf.numTrees, [150, 200, 300])
                     .addGrid(clf.maxDepth, [8, 10, 12])
                     .build())
    elif model_name == "gbt":
        clf = GBTClassifier(featuresCol="features", labelCol="label", maxIter=100, maxDepth=6, stepSize=0.1, seed=42)
        # Nota: GBT en Spark no soporta weightCol directamente antes de 3.4; se usa dataset balanceado/estratificado si es necesario.
        paramGrid = (ParamGridBuilder()
                     .addGrid(clf.maxIter, [60, 100])
                     .addGrid(clf.maxDepth, [5, 6, 8])
                     .build())
    else:
        raise ValueError("Modelo no soportado")

    pipeline = Pipeline(stages=[imputer] + indexers + encoders + [assembler, clf])
    evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderPR")

    tvs = TrainValidationSplit(estimator=pipeline, estimatorParamMaps=paramGrid,
                               evaluator=evaluator, trainRatio=0.8, parallelism=2)

    tvs_model = tvs.fit(train_w if model_name in ("lr","rf") else train_df)  # GBT sin weights
    return tvs_model

## 7) Evaluación: AUC, matriz y métricas derivadas

- Reportamos **AUC ROC** y **AUC PR**.

- Con el umbral 0.5 para referencia.

- Devuelve también las predicciones con la probabilidad `p_digital`.


In [ ]:
def evaluate_model(model, test_df, model_name="model"):
    pred = model.transform(test_df).withColumn("p_digital", vector_to_array("probability")[1]).cache()
    e_auc  = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
    e_aupr = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderPR")
    auc = e_auc.evaluate(pred)
    aupr = e_aupr.evaluate(pred)

    cm = (pred.groupBy("label","prediction").count().toPandas())
    tp = int(cm[(cm["label"]==1) & (cm["prediction"]==1)]["count"].sum())
    tn = int(cm[(cm["label"]==0) & (cm["prediction"]==0)]["count"].sum())
    fp = int(cm[(cm["label"]==0) & (cm["prediction"]==1)]["count"].sum())
    fn = int(cm[(cm["label"]==1) & (cm["prediction"]==0)]["count"].sum())

    accuracy  = (tp + tn) / max(tp + tn + fp + fn, 1)
    precision = tp / max(tp + fp, 1)
    recall    = tp / max(tp + fn, 1)
    f1        = (2 * precision * recall) / max(precision + recall, 1e-9)

    print(f"[{model_name}]  AUC ROC: {auc:.4f} | AUC PR: {aupr:.4f} | Acc: {accuracy:.4f} | Prec: {precision:.4f} | Rec: {recall:.4f} | F1: {f1:.4f}")
    return pred, {"auc_roc": auc, "auc_pr": aupr, "accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

## 8) Entrenamiento y comparación de modelos (LR, RF, GBT)

Seleccionamos el **mejor** por **AUC PR** (prioriza casos positivos).

In [ ]:
models = {}
metrics = {}
predictions = {}

for name in ["lr","rf","gbt"]:
    print(f"Entrenando modelo: {name}")
    m = build_and_fit(name, train)
    pred, mtx = evaluate_model(m, test, model_name=name)
    models[name] = m
    metrics[name] = mtx
    predictions[name] = pred.select("cliente_id","ym","label","p_digital")

# Selección por AUC PR
best_name = sorted(metrics.items(), key=lambda x: x[1]["auc_pr"], reverse=True)[0][0]
best_model = models[best_name]
best_pred  = predictions[best_name]

print("\n>>> Mejor modelo por AUC PR:", best_name, metrics[best_name])

## 9) Tuning de umbral (max F1)

Buscamos el umbral óptimo para el **mejor modelo**.


In [ ]:
def metrics_with_threshold_from_pred(pred_df, thr: float):
    p = pred_df.withColumn("pred_thr", (F.col("p_digital") >= F.lit(thr)).cast("int"))
    agg = p.agg(
        F.sum(F.when((F.col("label")==1) & (F.col("pred_thr")==1), 1).otherwise(0)).alias("tp"),
        F.sum(F.when((F.col("label")==0) & (F.col("pred_thr")==0), 1).otherwise(0)).alias("tn"),
        F.sum(F.when((F.col("label")==0) & (F.col("pred_thr")==1), 1).otherwise(0)).alias("fp"),
        F.sum(F.when((F.col("label")==1) & (F.col("pred_thr")==0), 1).otherwise(0)).alias("fn")
    ).first()
    tp, tn, fp, fn = [float(agg[x] or 0.0) for x in ("tp","tn","fp","fn")]
    total = tp + tn + fp + fn or 1.0
    accuracy  = (tp + tn) / total
    precision = tp / (tp + fp or 1.0)
    recall    = tp / (tp + fn or 1.0)
    f1        = (2 * precision * recall) / (precision + recall or 1e-9)
    return accuracy, precision, recall, f1

grid = [x/100 for x in range(30, 81, 5)]
rows = []
for t in grid:
    acc, pre, rec, f1 = metrics_with_threshold_from_pred(best_pred, t)
    rows.append((t, acc, pre, rec, f1))

thr_df = spark.createDataFrame(rows, ["threshold","accuracy","precision","recall","f1"]).orderBy(F.desc("f1"))
thr_df.show(20, False)

best_thr = thr_df.first()["threshold"]
acc, pre, rec, f1 = metrics_with_threshold_from_pred(best_pred, best_thr)
print(f"Mejor threshold: {best_thr:.2f} | Acc: {acc:.4f}  Prec: {pre:.4f}  Rec: {rec:.4f}  F1: {f1:.4f}")

## 10) Métricas por segmento (país / región)

Evalúa **dónde** funciona mejor el modelo (AUC por grupo). Se omiten segmentos con una sola clase.


In [ ]:
joined = (best_pred.join(test.select("cliente_id","ym","pais_cd","region_comercial_txt"),
                           on=["cliente_id","ym"], how="left"))

def auc_by_group(df_in, group_col: str):
    df_in = df_in.filter(F.col(group_col).isNotNull())
    groups = [r[0] for r in df_in.select(group_col).distinct().collect()]
    e_auc = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="p_digital", metricName="areaUnderROC")
    out = []
    for g in groups:
        subset = df_in.filter(F.col(group_col)==g)
        if subset.select("label").distinct().count() < 2:
            continue
        # Como 'p_digital' es probabilidad, usamos el evaluador con probabilidades: necesita una columna de rawPrediction.
        # Truco: crear una columna vector con forma [1-p, p] sólo para el evaluador.
        sub2 = subset.withColumn("rawPrediction", F.array(1.0 - F.col("p_digital"), F.col("p_digital")))
        auc_g = e_auc.evaluate(sub2)
        out.append((g, float(auc_g)))
    return spark.createDataFrame(out, [group_col, "auc_roc"]).orderBy(F.desc("auc_roc"))

print("AUC ROC por país:")
auc_by_group(joined, "pais_cd").show(truncate=False)

print("AUC ROC por región:")
auc_by_group(joined, "region_comercial_txt").show(truncate=False)

## 11) Calibración de probabilidad (curva de confiabilidad)

- Particionamos por **deciles** de probabilidad (`p_digital`).

- Mostramos **promedio predicho vs. observado** por bin.


In [ ]:
# ntile(10) sobre probabilidad
w_prob = Window.orderBy(F.col("p_digital").asc())
calib = (best_pred
         .withColumn("decile", F.ntile(10).over(w_prob))
         .groupBy("decile")
         .agg(F.avg("p_digital").alias("p_pred_mean"),
              F.avg(F.col("label").cast("double")).alias("p_obs_mean"),
              F.count("*").alias("n"))
         .orderBy("decile"))

calib.show(10, False)

# Plot opcional (si tienes matplotlib)
try:
    import matplotlib.pyplot as plt
    pdf = calib.toPandas()
    plt.figure(figsize=(5,4))
    plt.plot(pdf["p_pred_mean"], pdf["p_obs_mean"], marker="o")
    plt.plot([0,1],[0,1],"--")
    plt.title("Curva de calibración")
    plt.xlabel("Predicho")
    plt.ylabel("Observado")
    plt.tight_layout()
    plt.show()
except Exception as e:
    print("Plot no disponible (matplotlib no instalado):", e)

## 12) Backtesting (opcional)

Repite entrenamiento/evaluación en múltiples cortes temporales (`BACKTEST_SPLITS`) para verificar **estabilidad**.


In [ ]:
def run_split(ym_cut):
    train_bt = ds.filter(F.col("ym") < F.lit(ym_cut))
    test_bt  = ds.filter(F.col("ym") >= F.lit(ym_cut))
    m = build_and_fit(best_name, train_bt)
    pred_bt, mtx_bt = evaluate_model(m, test_bt, model_name=f"{best_name}@{ym_cut}")
    return ym_cut, mtx_bt

bt_rows = []
for cut in BACKTEST_SPLITS:
    try:
        ym_cut, mtx = run_split(cut)
        bt_rows.append((ym_cut, float(mtx["auc_pr"]), float(mtx["auc_roc"]), float(mtx["f1"])))
    except Exception as e:
        print(f"Backtest falló en corte {cut}:", e)

if bt_rows:
    bt_df = spark.createDataFrame(bt_rows, ["ym_cut","auc_pr","auc_roc","f1"]).orderBy("ym_cut")
    bt_df.show(truncate=False)

## 13) Exportar artefactos y métricas

- Guarda el **mejor modelo** (`models/`).

- Guarda **métricas** y **predicciones** (`results/`).

In [ ]:
# Guardar modelo
models_dir = f"models/{best_name}_next_digital_v3"
best_model.bestModel.write().overwrite().save(models_dir)

# Guardar métricas globales
from datetime import datetime
metrics_rows = [
    ("model", best_name),
    ("auc_pr", float(metrics[best_name]["auc_pr"])),
    ("auc_roc", float(metrics[best_name]["auc_roc"])),
    ("accuracy", float(metrics[best_name]["accuracy"])),
    ("precision", float(metrics[best_name]["precision"])),
    ("recall", float(metrics[best_name]["recall"])),
    ("f1", float(metrics[best_name]["f1"])),
    ("best_threshold", float(best_thr)),
    ("generated_at", datetime.utcnow().isoformat())
]
spark.createDataFrame([(k, v) for k, v in metrics_rows], ["metric","value"]).coalesce(1)      .write.mode("overwrite").json("results/metrics_v3")

# Guardar predicciones con probabilidad
best_pred.write.mode("overwrite").parquet("results/pred_test_v3")

print("Modelo guardado en:", models_dir)
print("Métricas y predicciones guardadas en 'results/'.")

## 14) Cierre de sesión Spark

In [ ]:
spark.stop()
print("Spark session stopped.")